In [1]:
import os
import json
import joblib
import time
import pandas as pd
import numpy as np
import random
from IPython.display import display, HTML
import torch

import nltk

# # Better pandas table
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

In [2]:
#connect to hugging face
!git clone https://github.com/huggingface/transformers \
&& cd transformers \
install
!pip install -q ./transformers

fatal: destination path 'transformers' already exists and is not an empty directory.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simplet5 0.1.4 requires transformers==4.16.2, but you have transformers 4.18.0.dev0 which is incompatible.


In [3]:
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [4]:
# Installing transformers
!pip install simpletransformers
#!pip install transformers
# Importing necessary libraries from simple transformers
from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs

In [7]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using: ', device)

Using:  cuda


In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

In [10]:
# path_models = '/content/drive/MyDrive/w266/models/'
# path_outputs = '/content/drive/MyDrive/w266/outputs/'
# path_processed_data= '/content/drive/MyDrive/w266/processed_data/'

In [11]:
path_models = './models/cnn_2e/'
path_outputs = './outputs/'
path_processed_data= './processed_data/'

In [15]:
data_name = 'cnn_test'
input_filename = data_name + '_prediction_v3_bart_cnn.json' 
#out_filename = data_name + '_summary_abs.json'

In [16]:

jsonFile = open(os.path.join(path_processed_data, input_filename), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()

details = {
    'headlines' : details_dict['headlines'],
    'text' : details_dict['text'],
    'keywords' : details_dict['keywords'],
    'summary_art' : details_dict['summary_art'],
    'entities' : details_dict['entities'],
#    'text_ner' : details_dict['text_ner'],
    'sentence_1s' : details_dict['sentence_1s'],
    'sentence_3s' : details_dict['sentence_3s'],
    'summary_ext' : details_dict['summary_ext'],
    'summary_abs' : details_dict['summary_abs'],
    'summary_extabs' : details_dict['summary_extabs'],
    'summary_ner' : details_dict['summary_ner'],
    'summary_t5' : details_dict['summary_t5'],
    'predict_1s' : details_dict['predict_1s'],
    'predict_3s' : details_dict['predict_3s'],
    'predict_text' : details_dict['predict_text'],
    'predict_ext' : details_dict['predict_ext'],
    'predict_abs' : details_dict['predict_abs'],
    'predict_extabs' : details_dict['predict_extabs'],
    'predict_art' : details_dict['predict_art'],
#    'predict_t5' : details_dict['predict_t5']
}

df_score = pd.DataFrame(details)

In [17]:
#print(len(details_dict['predict_ner']))

In [18]:
df_results = {k: details[k] for k in ('headlines', 'predict_text', 'predict_ext', 'predict_art', 'predict_3s', 'predict_abs', 'predict_extabs', 'predict_1s')}
df = pd.DataFrame(df_results)

In [19]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

import itertools
n = 9
new_dict = {}
first_n_pairs = itertools.islice(df_results.items(), n)
for key, value in first_n_pairs:
    #print(key, '::', value[3])
    new_dict[key] = value[424]

new_df = pd.DataFrame(new_dict.items(), columns=['Approach', 'Headlines'])
#new_df.style.set_properties(**{'text-align': 'left'})
new_df.style.set_properties(**{'text-align': 'left'}).set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])
new_df

,Approach,Headlines
0,headlines,Moneyball How businesses using data outsmart r...
1,predict_text,Big data revolution coming business Why big da...
2,predict_ext,Big data could save us jobs future Smart Busin...
3,predict_art,Big data could save IBM big data analytics ris...
4,predict_3s,Moneyball Moneyball How Billy Beane revolution...
5,predict_abs,Big data could save business life says McKinse...
6,predict_extabs,Big data could kill us all says Google CEO Tim...
7,predict_1s,Editor note Smart Business explores ways compa...


In [20]:
display(HTML(new_df.to_html(justify='left').replace('<tr>','<tr style="text-align: left;">')))

,Approach,Headlines
0,headlines,Moneyball How businesses using data outsmart rivals
1,predict_text,Big data revolution coming business Why big data could help us win big time .
2,predict_ext,Big data could save us jobs future Smart Business How digital revolution could help us thrive digital
3,predict_art,Big data could save IBM big data analytics risk falling behind experts say . .
4,predict_3s,Moneyball Moneyball How Billy Beane revolutionized baseball analytics ...
5,predict_abs,Big data could save business life says McKinsey report warns risk left behind us .
6,predict_extabs,Big data could kill us all says Google CEO Tim Cook ... .
7,predict_1s,Editor note Smart Business explores ways companies thinking smart thrive digitized world .


In [21]:
df_results = {k: details[k] for k in ('text', 'sentence_1s', 'sentence_3s', 'summary_ext', 'summary_abs', 'summary_extabs', 'summary_art')}
df = pd.DataFrame(df_results)

In [22]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

import itertools
n = 7
new_dict = {}
first_n_pairs = itertools.islice(df_results.items(), n)
for key, value in first_n_pairs:
    #print(key, '::', value[3])
    new_dict[key] = value[424]

new_df = pd.DataFrame(new_dict.items(), columns=['Summary', 'Content'])
#new_df.style.set_properties(**{'text-align': 'left'})
new_df.style.set_properties(**{'text-align': 'left'}).set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])
new_df

,Summary,Content
0,text,Editor note Smart Business explores ways compa...
1,sentence_1s,Editor note Smart Business explores ways compa...
2,sentence_3s,Editor note Smart Business explores ways compa...
3,summary_ext,Editor note Smart Business explores ways compa...
4,summary_abs,Smart Business explores ways companies thinkin...
5,summary_extabs,Smart Business explores ways companies thinkin...
6,summary_art,It called big data analytics ignore warned ris...


In [23]:
pip install Rouge

Note: you may need to restart the kernel to use updated packages.


In [24]:
from rouge import Rouge
rouge = Rouge()

In [25]:
score_1s = rouge.get_scores(details_dict['predict_1s'], details_dict['headlines'], avg=True)
df_results = {k: score_1s[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.272922,0.089370,0.241525
p,0.109200,0.030277,0.098448
f,0.142266,0.041741,0.126898


In [26]:
score_3s = rouge.get_scores(details_dict['predict_3s'], details_dict['headlines'], avg=True)
df_results = {k: score_3s[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.334828,0.120931,0.303272
p,0.183899,0.055403,0.166432
f,0.233341,0.074472,0.211162


In [27]:
score_text = rouge.get_scores(details_dict['predict_text'], details_dict['headlines'], avg=True)
df_results = {k: score_text[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.375520,0.136366,0.340279
p,0.200357,0.061691,0.181220
f,0.256962,0.083226,0.232504


In [28]:
score_ext = rouge.get_scores(details_dict['predict_ext'], details_dict['headlines'], avg=True)
df_results = {k: score_ext[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.358588,0.127085,0.325974
p,0.194610,0.057673,0.176530
f,0.247929,0.077685,0.225003


In [29]:
score_abs = rouge.get_scores(details_dict['predict_abs'], details_dict['headlines'], avg=True)
df_results = {k: score_abs[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.306080,0.098754,0.279966
p,0.161346,0.044632,0.147175
f,0.207808,0.060229,0.189691


In [30]:
score_extabs = rouge.get_scores(details_dict['predict_extabs'], details_dict['headlines'], avg=True)
df_results = {k: score_extabs[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.282819,0.088215,0.258372
p,0.149878,0.039929,0.136751
f,0.192700,0.053888,0.175878


In [31]:
# score_ner = rouge.get_scores(details_dict['predict_ner'], details_dict['headlines'], avg=True)
# df_results = {k: score_ner[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
# pd.DataFrame(df_results)

In [32]:
# score_t5 = rouge.get_scores(details_dict['predict_t5'], details_dict['headlines'], avg=True)
# df_results = {k: score_t5[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
# pd.DataFrame(df_results)

In [33]:
score_art = rouge.get_scores(details_dict['predict_art'], details_dict['headlines'], avg=True)
df_results = {k: score_art[k] for k in ('rouge-1', 'rouge-2', 'rouge-l')}
pd.DataFrame(df_results)

,rouge-1,rouge-2,rouge-l
r,0.406890,0.152974,0.368040
p,0.217203,0.068512,0.196059
f,0.278408,0.092671,0.251402


In [34]:
rouge_1 = {'baseline (1s)': {'r': score_1s['rouge-1']['r'], 'p':score_1s['rouge-1']['p'], 'f': score_1s['rouge-1']['f']}, 
        'baseline (3s)': {'r': score_3s['rouge-1']['r'], 'p':score_3s['rouge-1']['p'], 'f': score_3s['rouge-1']['f']}, 
           'text': {'r': score_text['rouge-1']['r'], 'p':score_text['rouge-1']['p'], 'f': score_text['rouge-1']['f']},
           'ext': {'r': score_ext['rouge-1']['r'], 'p':score_ext['rouge-1']['p'], 'f': score_ext['rouge-1']['f']},
           'abs': {'r': score_abs['rouge-1']['r'], 'p':score_abs['rouge-1']['p'], 'f': score_abs['rouge-1']['f']},
           'extabs': {'r': score_extabs['rouge-1']['r'], 'p':score_extabs['rouge-1']['p'], 'f': score_extabs['rouge-1']['f']},
           'artsum': {'r': score_art['rouge-1']['r'], 'p':score_art['rouge-1']['p'], 'f': score_art['rouge-1']['f']},
#           't5': {'r': score_t5['rouge-1']['r'], 'p':score_t5['rouge-1']['p'], 'f': score_t5['rouge-1']['f']},
}

In [35]:
df_results = {k: rouge_1[k] for k in ('artsum', 'text', 'ext', 'baseline (3s)',  'baseline (1s)', 'abs', 'extabs', )}
pd.DataFrame(df_results)

,artsum,text,ext,baseline (3s),baseline (1s),abs,extabs
r,0.406890,0.375520,0.358588,0.334828,0.272922,0.306080,0.282819
p,0.217203,0.200357,0.194610,0.183899,0.109200,0.161346,0.149878
f,0.278408,0.256962,0.247929,0.233341,0.142266,0.207808,0.192700


In [36]:
rouge_2 = {'baseline (1s)': {'r': score_1s['rouge-2']['r'], 'p':score_1s['rouge-2']['p'], 'f': score_1s['rouge-2']['f']}, 
        'baseline (3s)': {'r': score_3s['rouge-2']['r'], 'p':score_3s['rouge-2']['p'], 'f': score_3s['rouge-2']['f']}, 
           'text': {'r': score_text['rouge-2']['r'], 'p':score_text['rouge-2']['p'], 'f': score_text['rouge-2']['f']},
           'ext': {'r': score_ext['rouge-2']['r'], 'p':score_ext['rouge-2']['p'], 'f': score_ext['rouge-2']['f']},
           'abs': {'r': score_abs['rouge-2']['r'], 'p':score_abs['rouge-2']['p'], 'f': score_abs['rouge-2']['f']},
           'extabs': {'r': score_extabs['rouge-2']['r'], 'p':score_extabs['rouge-2']['p'], 'f': score_extabs['rouge-2']['f']},
           'artsum': {'r': score_art['rouge-2']['r'], 'p':score_art['rouge-2']['p'], 'f': score_art['rouge-2']['f']},
#           't5': {'r': score_t5['rouge-2']['r'], 'p':score_t5['rouge-2']['p'], 'f': score_t5['rouge-2']['f']},
}
df_results = {k: rouge_2[k] for k in ('artsum', 'text', 'ext', 'baseline (3s)',  'baseline (1s)', 'abs', 'extabs', )}
pd.DataFrame(df_results)

,artsum,text,ext,baseline (3s),baseline (1s),abs,extabs
r,0.152974,0.136366,0.127085,0.120931,0.089370,0.098754,0.088215
p,0.068512,0.061691,0.057673,0.055403,0.030277,0.044632,0.039929
f,0.092671,0.083226,0.077685,0.074472,0.041741,0.060229,0.053888


In [37]:
rouge_l = {'baseline (1s)': {'r': score_1s['rouge-l']['r'], 'p':score_1s['rouge-l']['p'], 'f': score_1s['rouge-l']['f']}, 
        'baseline (3s)': {'r': score_3s['rouge-l']['r'], 'p':score_3s['rouge-l']['p'], 'f': score_3s['rouge-l']['f']}, 
           'text': {'r': score_text['rouge-l']['r'], 'p':score_text['rouge-l']['p'], 'f': score_text['rouge-l']['f']},
           'ext': {'r': score_ext['rouge-l']['r'], 'p':score_ext['rouge-l']['p'], 'f': score_ext['rouge-l']['f']},
           'abs': {'r': score_abs['rouge-l']['r'], 'p':score_abs['rouge-l']['p'], 'f': score_abs['rouge-l']['f']},
           'extabs': {'r': score_extabs['rouge-l']['r'], 'p':score_extabs['rouge-l']['p'], 'f': score_extabs['rouge-l']['f']},
           'artsum': {'r': score_art['rouge-l']['r'], 'p':score_art['rouge-l']['p'], 'f': score_art['rouge-l']['f']},
#           't5': {'r': score_t5['rouge-l']['r'], 'p':score_t5['rouge-l']['p'], 'f': score_t5['rouge-l']['f']},
}
df_results = {k: rouge_l[k] for k in ('artsum', 'text', 'ext', 'baseline (3s)',  'baseline (1s)', 'abs', 'extabs', )}
pd.DataFrame(df_results)

,artsum,text,ext,baseline (3s),baseline (1s),abs,extabs
r,0.368040,0.340279,0.325974,0.303272,0.241525,0.279966,0.258372
p,0.196059,0.181220,0.176530,0.166432,0.098448,0.147175,0.136751
f,0.251402,0.232504,0.225003,0.211162,0.126898,0.189691,0.175878


In [38]:
# Evaluation Metrics - BLEU implementation
from nltk.translate.bleu_score import sentence_bleu


In [39]:
# Function to calculate the score
def calc_bleu(x, y):
    L = 0
    for i, j in zip(x, y):
      L += sentence_bleu([i], j, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=None,  auto_reweigh=False,)

    # Average BLEU score of whole corpuses
    return round(L/len(x), 4)



In [40]:
bleu_1s = calc_bleu(details_dict['headlines'], details_dict['predict_1s'])
print("BLEU score: {:.04f}".format(bleu_1s))

/home/ubuntu/anaconda3/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/anaconda3/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/anaconda3/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use Smoo

BLEU score: 0.1328


In [41]:
bleu_3s = calc_bleu(details_dict['headlines'], details_dict['predict_3s'])
print("BLEU score: {:.04f}".format(bleu_3s))

BLEU score: 0.2180


In [42]:
bleu_text = calc_bleu(details_dict['headlines'], details_dict['predict_text'])
print("BLEU score: {:.04f}".format(bleu_text))

BLEU score: 0.2405


In [43]:
bleu_ext = calc_bleu(details_dict['headlines'], details_dict['predict_ext'])
print("BLEU score: {:.04f}".format(bleu_ext))

BLEU score: 0.2322


In [44]:
bleu_abs = calc_bleu(details_dict['headlines'], details_dict['predict_abs'])
print("BLEU score: {:.04f}".format(bleu_abs))

BLEU score: 0.2076


In [45]:
bleu_extabs = calc_bleu(details_dict['headlines'], details_dict['predict_extabs'])
print("BLEU score: {:.04f}".format(bleu_extabs))

BLEU score: 0.1960


In [46]:
# bleu_ner = calc_bleu(details_dict['headlines'], details_dict['predict_ner'])
# print("BLEU score: {:.04f}".format(bleu_ner))

In [47]:
# bleu_t5 = calc_bleu(details_dict['headlines'], details_dict['predict_t5'])
# print("BLEU score: {:.04f}".format(bleu_t5))

In [48]:
bleu_art = calc_bleu(details_dict['headlines'], details_dict['predict_art'])
print("BLEU score: {:.04f}".format(bleu_art))

BLEU score: 0.2512


In [49]:
bleu_score = {'baseline (1s)': {'bleu_score': bleu_1s}, 
                'baseline (3s)': {'bleu_score': bleu_3s}, 
    'text' : {'bleu_score': bleu_text}, 
    'ext' : {'bleu_score': bleu_ext},
    'abs' : {'bleu_score': bleu_abs},
    'extabs' : {'bleu_score': bleu_extabs},
    'artsum' : {'bleu_score': bleu_art},
#    't5' : {'bleu_score': bleu_t5}
}

In [50]:
df_results = {k: bleu_score[k] for k in ('artsum', 'text', 'ext', 'baseline (3s)',  'abs', 'extabs', 'baseline (1s)')}
pd.DataFrame(df_results)

,artsum,text,ext,baseline (3s),abs,extabs,baseline (1s)
bleu_score,0.2512,0.2405,0.2322,0.218,0.2076,0.196,0.1328


In [51]:
pip install bert-score

Note: you may need to restart the kernel to use updated packages.


In [52]:
from bert_score import score

In [53]:
# Baseline
P, R, F1 = score(details_dict['predict_1s'], details_dict['headlines'], lang="en", verbose=True)
F1_1s = F1.mean().item()
P_1s = P.mean().item()
R_1s = R.mean().item()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
# Ext
P, R, F1 = score(details_dict['predict_3s'], details_dict['headlines'], lang="en", verbose=True)
F1_3s = F1.mean().item()
P_3s = P.mean().item()
R_3s = R.mean().item()


In [ ]:
# Abs
P, R, F1 = score(details_dict['predict_text'], details_dict['headlines'], lang="en", verbose=True)
F1_text = F1.mean().item()
P_text = P.mean().item()
R_text = R.mean().item()

In [ ]:
# ExtAbs
P, R, F1 = score(details_dict['predict_ext'], details_dict['headlines'], lang="en", verbose=True)
F1_ext = F1.mean().item()
P_ext = P.mean().item()
R_ext = R.mean().item()

In [ ]:
P, R, F1 = score(details_dict['predict_abs'], details_dict['headlines'], lang="en", verbose=True)
F1_abs = F1.mean().item()
P_abs = P.mean().item()
R_abs = R.mean().item()

In [ ]:
P, R, F1 = score(details_dict['predict_extabs'], details_dict['headlines'], lang="en", verbose=True)
F1_extabs = F1.mean().item()
P_extabs = P.mean().item()
R_extabs = R.mean().item()

In [ ]:
# P, R, F1 = score(details_dict['predict_ner'], details_dict['headlines'], lang="en", verbose=True)
# F1_ner = F1.mean().item()
# P_ner = P.mean().item()
# R_ner = R.mean().item()

In [ ]:
# P, R, F1 = score(details_dict['predict_t5'], details_dict['headlines'], lang="en", verbose=True)
# F1_t5 = F1.mean().item()
# P_t5 = P.mean().item()
# R_t5 = R.mean().item()

In [ ]:
P, R, F1 = score(details_dict['predict_art'], details_dict['headlines'], lang="en", verbose=True)
F1_art = F1.mean().item()
P_art = P.mean().item()
R_art = R.mean().item()

In [ ]:
# import matplotlib.pyplot as plt
# plt.hist(F1, bins=20)
# plt.show()
# from bert_score import plot_example

# p1 = P1[0]
# y = Y[0]
# plot_example(p1, y, lang="en")

In [ ]:
# bert_score = {'baseline': {'P': P_base,
#   'R': R_base,
#   'F1': F1_base}, 
#    'ext' : {'P': P_ext,
#   'R': R_ext,
#   'F1': F1_ext}, 
#   'abs' : {'P': P_abs,
#   'R': R_abs,
#   'F1': F1_abs},
#   'extabs' : {'P': P_extabs,
#   'R': R_extabs,
#   'F1': F1_extabs},
# }

bert_score = {'P': {  'art': P_art,
                      'text': P_text,
                      'ext': P_ext,
  'baseline (3 sentences)': P_3s,

  'abs': P_abs,
  'extabs': P_extabs,
'baseline (1 sentence)': P_1s,}, 
   'R' : {  'art': R_art,
            'text': R_text,
            'ext': R_ext,
  'baseline (3 sentences)': R_3s,
  'abs': R_abs,
  'extabs': R_extabs,
   'baseline (1 sentence)': R_1s,}, 
  'F1' : {  'art': F1_art,
            'text': F1_text,
            'ext': F1_ext,
  'baseline (3 sentences)': F1_3s,
  'abs': F1_abs,
  'extabs': F1_extabs,
  'baseline (1 sentence)': F1_1s,},
}



In [ ]:
df_results = {k: bert_score[k] for k in ('P', 'R', 'F1')}
pd.DataFrame(df_results)

In [ ]:
#!pip install --upgrade pip  # ensures that pip is current
#!git clone https://github.com/google-research/bleurt.git
#!cd bleurt
#!pip install .

In [ ]:
# import unittest

# python -m unittest bleurt.score_test
# python -m unittest bleurt.score_not_eager_test
# python -m unittest bleurt.finetune_test
# python -m unittest bleurt.score_files_test

In [ ]:
from functools import reduce
  
def average(lst):
    return reduce(lambda a, b: a + b, lst) / len(lst)

In [ ]:
from bleurt import score

In [ ]:
checkpoint = "bleurt/bleurt/test_checkpoint"
scorer = score.BleurtScorer(checkpoint)

In [ ]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_1s']) 
bleurt_1s = average(scores)

In [ ]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_3s']) 
bleurt_3s = average(scores)

In [ ]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_text']) 
bleurt_text = average(scores)

In [ ]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_ext']) 
bleurt_ext = average(scores)

In [ ]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_abs']) 
bleurt_abs = average(scores)

In [ ]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_extabs']) 
bleurt_extabs = average(scores)

In [ ]:
# scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_ner']) 
# bleurt_ner = average(scores)

In [ ]:
# scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_t5']) 
# bleurt_t5 = average(scores)

In [ ]:
scores = scorer.score(references=details_dict['headlines'], candidates=details_dict['predict_art']) 
bleurt_art = average(scores)

In [ ]:
bleurt_score = {'baseline (1s)': {'bleurt_score': bleurt_1s}, 
                'baseline (3s)': {'bleurt_score': bleurt_3s}, 
    'text' : {'bleurt_score': bleurt_text}, 
    'ext' : {'bleurt_score': bleurt_ext},
    'abs' : {'bleurt_score': bleurt_abs},
    'extabs' : {'bleurt_score': bleurt_extabs},
#    'ner' : {'bleurt_score': bleurt_ner},
#    't5' : {'bleurt_score': bleurt_t5},
    'art' : {'bleurt_score': bleurt_art},                
}

# bleurt_score = {'bleurt_score': {'baseline': bleurt_baseline}, 
#    'bleurt_score' : {'ext': bleurt_ext}, 
#   'bleurt_score' : {'abs': bleurt_abs},
#   'bleurt_score' : {'extabs': bleurt_extabs}
# }

#bleurt_score = {'bleurt_score': {'baseline': bleurt_baseline, 'ext': bleurt_ext, 'abs': bleurt_abs, 'extabs': bleurt_extabs}}

In [ ]:
#df_results = {k: bleurt_score[k] for k in ('bleurt_score')}
df_results = {k: bleurt_score[k] for k in ('art', 'text', 'ext', 'baseline (3s)', 'abs', 'extabs', 'baseline (1s)')}
pd.DataFrame(df_results)

In [ ]:
#score = nltk.translate.meteor_score.meteor_score(reference, model)
# print (nltk.translate.meteor_score.meteor_score(
#     ["this is an apple", "that is an apple"], "an apple on this tree"))
# print (nltk.translate.meteor_score.meteor_score(
#     ["this is an apple", "that is an apple"], "a red color fruit"))
# nltk.translate.meteor_score.meteor_score(P1, "a red color fruit")

In [ ]:
# meteor_score_1s = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_1s']):
#     meteor_score_1s += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_1s = meteor_score_1s/len(details_dict['headlines'])
# meteor_score_1s

In [ ]:
# meteor_score_3s = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_3s']):
#     meteor_score_3s += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_3s = meteor_score_3s/len(details_dict['headlines'])
# meteor_score_3s

In [ ]:
# meteor_score_text = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_text']):
#     meteor_score_text += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_text = meteor_score_text/len(details_dict['headlines'])
# meteor_score_text

In [ ]:
# meteor_score_ext = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_ext']):
#     meteor_score_ext += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_ext = meteor_score_ext/len(details_dict['headlines'])
# meteor_score_ext

In [ ]:
# meteor_score_abs = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_abs']):
#     meteor_score_abs += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_abs = meteor_score_abs/len(details_dict['headlines'])
# meteor_score_abs

In [ ]:
# meteor_score_extabs = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_extabs']):
#     meteor_score_extabs += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_extabs = meteor_score_extabs/len(details_dict['headlines'])
# meteor_score_extabs

In [ ]:
# meteor_score_ner = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_ner']):
#     meteor_score_ner += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_ner = meteor_score_ner/len(details_dict['headlines'])
# meteor_score_ner

In [ ]:
# meteor_score_t5 = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_t5']):
#     meteor_score_t5 += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_t5 = meteor_score_t5/len(details_dict['headlines'])
# meteor_score_t5

In [ ]:
# meteor_score_art = 0
# for x, y in zip(details_dict['headlines'], details_dict['predict_art']):
#     meteor_score_art += nltk.translate.meteor_score.meteor_score(x, y)
# meteor_score_art = meteor_score_art/len(details_dict['headlines'])
# meteor_score_art

In [ ]:
# meteor_score = {'baseline (1s)': {'meteor_score': meteor_score_1s}, 
#                 'baseline (3s)': {'meteor_score': meteor_score_3s}, 
#     'text' : {'meteor_score': meteor_score_text}, 
#     'ext' : {'meteor_score': meteor_score_ext},
#     'abs' : {'meteor_score': meteor_score_abs},
#     'extabs' : {'meteor_score': meteor_score_extabs},
#     'ner' : {'meteor_score': meteor_score_ner},
#     't5' : {'meteor_score': meteor_score_t5},
#     'art' : {'meteor_score': meteor_score_art}
# }
# df_results = {k: meteor_score[k] for k in ('baseline (1s)', 'baseline (3s)', 'text', 'ext', 'abs', 'extabs', 'ner', 't5', 'art')}
# pd.DataFrame(df_results)

In [ ]:
len(details_dict['headlines'])